In [2]:
import os, sys, shutil, glob, argparse
import numpy as np
from PIL import Image
from lxml import etree
import cv2
import pandas as pd

In [30]:
voc="./VOCHandsDataset/"

def make_directories():
    if os.path.exists(voc):
        prompt=raw_input('Directory already exists. Overwrite? (yes, no): ')
        if prompt == 'no':
            exit(0)
        shutil.rmtree(voc)
        
    os.makedirs(voc)
    os.makedirs(voc + "Annotations")
    os.makedirs(voc + "ImageSets")
    os.makedirs(voc + "ImageSets/Layout")
    os.makedirs(voc + "JPEGImages")

In [41]:
def write_voc_file(fname, label, coord, img_width, img_height):
    num_hands = len(coord)
    annotation = etree.Element('annotation')
    filename = etree.Element('filename')
    filename.text = fname
    annotation.append(filename)
    folder = etree.Element('folder')
    folder.text = "Hands_Data"
    annotation.append(folder)
    for ix in range(num_hands):
        object = etree.Element('object')
        annotation.append(object)
        name = etree.Element('name')
        name.text = label
        object.append(name)
        bndbox = etree.Element('bndbox')
        object.append(bndbox)
        xmax = etree.Element('xmax')
        xmax.text = str(coord[ix][2])
        bndbox.append(xmax)
        xmin = etree.Element('xmin')
        xmin.text = str(coord[ix][0])
        bndbox.append(xmin)
        ymax = etree.Element('ymax')
        ymax.text = str(coord[ix][3])
        bndbox.append(ymax)
        ymin = etree.Element('ymin')
        ymin.text = str(coord[ix][1])
        bndbox.append(ymin)
        difficult = etree.Element('difficult')
        difficult.text = '0'
        object.append(difficult)
        occluded = etree.Element('occluded')
        occluded.text = '0'
        object.append(occluded)
        pose = etree.Element('pose')
        pose.text = 'Unspecified'
        object.append(pose)
        truncated = etree.Element('truncated')
        truncated.text = '1'
        object.append(truncated)
    img_size = etree.Element('size')
    annotation.append(img_size)
    depth = etree.Element('depth')
    depth.text = '3'
    img_size.append(depth)
    height = etree.Element('height')
    height.text = str(img_height)
    img_size.append(height)
    width = etree.Element('width')
    width.text = str(img_width)
    img_size.append(width)

    return annotation

In [42]:
test_data = pd.read_csv('./csvs/test_labels.csv')
test_matrix = test_data.values
print test_matrix.shape
test_data.head(n=4)

(1498, 8)


,filename,width,height,class,xmin,ymin,xmax,ymax
0,CARDS_COURTYARD_B_T_frame_0206.jpg,1280,720,hand,639,439,824,547
1,CARDS_COURTYARD_B_T_frame_0206.jpg,1280,720,hand,477,362,596,487
2,CARDS_COURTYARD_B_T_frame_0647.jpg,1280,720,hand,587,380,763,482
3,CARDS_COURTYARD_B_T_frame_0647.jpg,1280,720,hand,437,336,584,465


In [43]:
def generate_dataset(data):
    count = 0
    cnt = 0
    #make_directories()
    data_dictionary = dict()
    count_dictionary = dict()
    for ix in range(data.shape[0]):
    #     if count>2:
    #         break
        count+=1
        sample = data.loc[ix]
        filename = sample['filename'].split('.jpg')[0]
        img_width = sample['width']
        img_height = sample['height']
        label = sample['class']
        coord_val = [sample['xmin'], sample['ymin'], sample['xmax'], sample['ymax']]

        if filename not in data_dictionary:
            data_dictionary[filename] = dict()
            count_dictionary[filename] = 0        
            
            data_dictionary[filename]['label'] = label
            data_dictionary[filename]['img_height'] = img_height
            data_dictionary[filename]['img_width'] = img_width
            data_dictionary[filename]['coords'] = []
            data_dictionary[filename]['coords'].append(coord_val)
        
        else:
            data_dictionary[filename]['coords'].append(coord_val)
        
    return data_dictionary

def generate_xml(data_dictionary):
    cnt = 0
    for ix in data_dictionary.keys():
        filename = ix
        coord = data_dictionary[ix]['coords']
        label = data_dictionary[ix]['label']
        img_height = data_dictionary[ix]['img_height']
        img_width = data_dictionary[ix]['img_width']
        coord = data_dictionary[ix]['coords']
        annotation = write_voc_file(fname=filename, label=label, img_height=img_height, img_width=img_width,
                               coord=coord)

        et = etree.ElementTree(annotation)
        xml_path = voc + "Annotations/" + filename + ".xml"
        if os.path.exists(xml_path):
            print 'over writing'
        et.write(voc + "Annotations/" + filename + ".xml", pretty_print=True)                    
        cnt+=1
        if(cnt%100==0):
            print(str(cnt)+" files completed...")
    #print "Total count="+str(cnt)

In [45]:
train_data = pd.read_csv('./csvs/train_labels.csv')
data_dictionary = generate_dataset(train_data)
generate_xml(data_dictionary)
print len(os.listdir(voc + 'Annotations/'))

over writing
over writing
over writing
over writing
over writing
100 files completed...
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
200 files completed...
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
300 files completed...
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
400 files completed...
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
500 files completed...
over writing
over writing
over writing
over writing
over writing
600 files completed...
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
over writing
700 files completed...
over writing
over writing
over writing
over writing
over writing
over writing
over wr

In [15]:
test_data.loc[0]

filename    CARDS_COURTYARD_B_T_frame_0206.jpg
width                                     1280
height                                     720
class                                     hand
xmin                                       639
ymin                                       439
xmax                                       824
ymax                                       547
Name: 0, dtype: object

In [49]:
#os.mkdir('../VOC_inhousehands')
#os.mkdir('../VOC_inhousehands/Annotations')
os.mkdir('../VOC_inhousehands/JPEGImages')

In [6]:
def generate_inhouse_dataset(fname, label, coord, img_height, img_width):
    annotation = etree.Element('annotation')
    filename = etree.Element('filename')
    filename.text = fname
    annotation.append(filename)
    folder = etree.Element('folder')
    folder.text = "DeepFashion"
    annotation.append(folder)
    object = etree.Element('object')
    annotation.append(object)
    name = etree.Element('name')
    name.text = label
    object.append(name)
    bndbox = etree.Element('bndbox')
    object.append(bndbox)
    xmax = etree.Element('xmax')
    xmax.text = str(coord[2])
    bndbox.append(xmax)
    xmin = etree.Element('xmin')
    xmin.text = str(coord[0])
    bndbox.append(xmin)
    ymax = etree.Element('ymax')
    ymax.text = str(coord[3])
    bndbox.append(ymax)
    ymin = etree.Element('ymin')
    ymin.text = str(coord[1])
    bndbox.append(ymin)
    difficult = etree.Element('difficult')
    difficult.text = '0'
    object.append(difficult)
    occluded = etree.Element('occluded')
    occluded.text = '0'
    object.append(occluded)
    pose = etree.Element('pose')
    pose.text = 'Unspecified'
    object.append(pose)
    truncated = etree.Element('truncated')
    truncated.text = '1'
    object.append(truncated)
    img_size = etree.Element('size')
    annotation.append(img_size)
    depth = etree.Element('depth')
    depth.text = '3'
    img_size.append(depth)
    height = etree.Element('height')
    height.text = str(img_height)
    img_size.append(height)
    width = etree.Element('width')
    width.text = str(img_width)
    img_size.append(width)

    return annotation

In [4]:
def get_coord_list(coords):
    xmin, ymin, xmax, ymax = coords.split('[')[-1].split(']')[0].split(',')
    coord_list = [int(xmin), int(ymin), int(xmax), int(ymax)]
    return coord_list
print get_coord_list('[2, 3, 4, 5]')

[2, 3, 4, 5]


In [7]:
folds = os.listdir('../Renamed_Gesture_Frames/')

inhouse_dict = dict()
for fld in folds:
    key, value = '_'.join(fld.split('_')[:-1]), fld.split('_')[-1]
    inhouse_dict[key] = value

log_f = open('./Log_FileMechdata.txt', 'w+')
#log_fl = open('../InHouse_LogFile.txt', 'w+')
#log_fl2 = open('../InHouse_LogFile2.txt', 'w+')
imgs = os.listdir('../Combined_Hands/')
cnt = 0
for ig in imgs:
    coords = inhouse_dict['_'.join(ig.split('_')[:-1])]
    
    try:
        coord_list = get_coord_list(coords)
        label = 'hand'
        filename = ig.split('.jpg')[0]
        img_height, img_width, _ = np.array(Image.open('../Combined_Hands/'+ ig)).shape
    except:
        log_f.write(ig + '\n')
        continue
        
    annotation = generate_inhouse_dataset(filename, label, coord_list, img_height, img_width)
    et = etree.ElementTree(annotation)
    xml_path = '../VOC_inhousehands/' + "Annotations/" + filename + ".xml"
    #if os.path.exists(xml_path):
        #print 'over writing'
    et.write('../VOC_inhousehands/' + "Annotations/" + filename + ".xml", pretty_print=True)                    
    cnt+=1
    if(cnt%100==0):
        print(str(cnt)+" files completed...")
    
#     try:
#         coord_list = get_coord_list(coords)
#     except:
#         log_fl.write(ig + '\n')
#         continue
#     label = 'hand'
#     filename = ig.split('.jpg')[0]
#     try:
#         img_height, img_width, _ = np.array(Image.open('../Combined_Hands/'+ ig)).shape
#     except:
#         log_fl2.write(ig+'\n')
#         continue
#     annotation = generate_inhouse_dataset(filename, label, coord_list, img_height, img_width)
#     et = etree.ElementTree(annotation)
#     xml_path = '../VOC_inhousehands/' + "Annotations/" + filename + ".xml"
#     #if os.path.exists(xml_path):
#         #print 'over writing'
#     et.write('../VOC_inhousehands/' + "Annotations/" + filename + ".xml", pretty_print=True)                    
#     cnt+=1
#     if(cnt%100==0):
#         print(str(cnt)+" files completed...")

100 files completed...
200 files completed...
300 files completed...
400 files completed...
500 files completed...
600 files completed...
700 files completed...
800 files completed...
900 files completed...
1000 files completed...
1100 files completed...
1200 files completed...
1300 files completed...
1400 files completed...
1500 files completed...
1600 files completed...
1700 files completed...
1800 files completed...
1900 files completed...
2000 files completed...
2100 files completed...
2200 files completed...
2300 files completed...
2400 files completed...
2500 files completed...
2600 files completed...
2700 files completed...
2800 files completed...
2900 files completed...
3000 files completed...
3100 files completed...
3200 files completed...
3300 files completed...
3400 files completed...
3500 files completed...
3600 files completed...
3700 files completed...
3800 files completed...
3900 files completed...
4000 files completed...
4100 files completed...
4200 files completed...
4